In [53]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import random as rd
from matplotlib.gridspec import GridSpec
from matplotlib import patches

In [54]:
N = 1000
sinyal_input = np.zeros(2*N+1)

def DFT(x,N,fs):
    x_real = np.zeros(N)
    x_imaj = np.zeros(N)
    magDFT = np.zeros(N)
    for k in range(N):
        for n in range(N):
            x_real[k] += x[n]*np.cos(2*np.pi*k*n/N)
            x_imaj[k] += x[n]*np.sin(2*np.pi*k*n/N)

    for k in range(N):
        magDFT[k] = np.sqrt(np.square(x_real[k]) + np.square(x_imaj[k]))/(fs/2)

    return magDFT

def input_data(a1='2', f1='25', a2='0.5', f2='150', a3='2', f3='250', fs='1000'):
    a1 = int(a1)
    f1 = int(f1)
    a2 = float(a2)
    f2 = float(f2)
    a3 = float(a3)
    f3 = float(f3)
    fs = float(fs)
    
    m = 0
    std = 0.7
    noise = rd.gauss(m,std)
    n = np.arange(0,N)
    global sinyal_input 
    sinyal_input = a1*np.sin(2*np.pi*n*f1/fs)+a2*np.sin(2*np.pi*n*f2/fs)+a3*np.cos(2*np.pi*n*f3/fs)+noise

    ## DFT
    global x_DFT
    x_DFT = DFT(sinyal_input,N,fs)
    
    fig = plt.figure(constrained_layout=True, figsize=(15,4))

    gs = GridSpec(1, 6, figure=fig)
    ax1 = fig.add_subplot(gs[0, :4])
    ax2 = fig.add_subplot(gs[0, 4:])

    ## Sinyal Input
    ax1.plot(n ,sinyal_input, color='blue', linewidth=1.0)
    ax1.set_title('Sinyal Input', fontsize='xx-large')
    ax1.set_xlabel('Time (s)', fontsize='x-large')
    ax1.set_ylabel('Amplituo', fontsize='x-large')
    ax1.grid()

    ## Response Frequency
    i = np.arange(1, int(round(fs/2)))
    ax2.plot(i, x_DFT[i], color='blue', linewidth=1.0)
    ax2.set_title('Respon Frekuensi Sinyal Input', fontsize='xx-large')
    ax2.set_xlabel('Frekuensi[Hz]', fontsize='x-large')
    ax2.set_ylabel('Magnitudo', fontsize='x-large')
    ax2.grid()

In [55]:
widgets.interact(input_data, a1='2', f1='25', a2='0.5', f2='150', a3='1', f3='250', fs='1000')

interactive(children=(Text(value='2', description='a1'), Text(value='25', description='f1'), Text(value='0.5',…

<function __main__.input_data(a1='2', f1='25', a2='0.5', f2='150', a3='2', f3='250', fs='1000')>

In [56]:
def wham (n,M):
    0.54 + 0.46*np.cos((2*np.pi*n)/M)

def wblack (n,M):
    0.42 + 0.5*np.cos((np.pi*n)/M) + 0.08*np.cos((2*np.pi*n)/M)

In [62]:
fil = widgets.RadioButtons(
    options=['LPF', 'HPF', 'BPF','BSF'],
    value='LPF',
    description='Filter'
)

win = widgets.RadioButtons(
    options=['Rectangular', 'Hamming', 'Blackman'],
    value='Rectangular',
    description='Window'
)

def fir (filter='LPF', window='Rectangular', M=6, fc=30, fl=0, fh=0):
    # Set Value
    ohm_cutoff = 2*np.pi*fc/Fs
    ohm_h = 2*np.pi*fh/Fs
    ohm_l = 2*np.pi*fl/Fs
        
    # Response Impulse w(i)
    w = np.zeros(2*M+1)
    for i in range(-M, M+1):
        if window == 'Rectangular':
            w[i] = 1
        elif window == 'Hamming':
            w[i] = 0.54 + 0.46*np.cos((np.pi*i)/M)
        elif window == 'Blackman':
            w[i] = 0.42 + 0.5*np.cos((np.pi*i)/M) + 0.08*np.cos((2*np.pi*i)/M)
    
    # Impulse Respose hw(i)
    hw = np.zeros(2*M+1)
    h = np.zeros(2*M+1)
    cont = 0
    for i in range(-M, M+1):
        if filter == 'LPF': #LPF
            if i == 0:
                h[i] = ohm_cutoff/np.pi
            else :
                h[i] = np.sin(ohm_cutoff*i)/(np.pi*i)
        elif filter == 'HPF': #HPF
            if i == 0:
                h[i] = (np.pi-ohm_cutoff)/np.pi
            else :
                h[i] = -np.sin(ohm_cutoff*i)/(np.pi*i)
        elif filter == 'BPF': #BPF
            if i == 0:
                h[i] = (ohm_h-ohm_l)/np.pi
            else :
                h[i] = (np.sin(ohm_h*i)-np.sin(ohm_l*i))/(np.pi*i)
        elif filter == 'BSF':
            if i == 0:
                h[i] = (np.pi-ohm_h+ohm_l)/np.pi
            else :
                h[i] = -(np.sin(ohm_h*i)-np.sin(ohm_l*i))/(np.pi*i)
        else : 
            break
            
    
    hw = h*w
    
    # Response Frequency Hw(ohm)
    hw_dft = np.zeros(360)
    
    for i in range(-180,181):
        cont = 0
        ohm = i*np.pi/180
        for j in range(1, M):
            cont += hw[j]*np.cos(j*ohm)
            
        hw_dft[i] = hw[0] + 2*cont    
    
    
    # Response Frequency Hw dalam dB
    hw_dft_db = np.zeros(360)
    for i in range(-180, 181):
        hw_dft_db[i] = 20*np.log10(abs((hw_dft[i])/np.max(hw_dft)))
        
        if hw_dft_db[i] <-80:
            hw_dft_db[i] = -80
            
    # The Output
    hn = np.zeros(2*M+1)
    y_forward = np.zeros(2*N+1)
    y_backward = np.zeros(2*N+1)
    noise = np.zeros(2*N+1)
    
    for i in range(2*M+1):
        hn[i] = h[i-M]
        
    for i in range(N): #forward
        for j in range(2*M+1):
            y_forward[i] += x[i-j]*hn[j]
            
    for i in range(N-1, -1, -1): #backward
        for j in range(2*M+1):
            y_backward[i] += y_forward[i+j]*hn[j]
            
        noise[i] = x[i]-y_backward[i]

        
    # DFT filtered Signal
    output_dft = dft(y_backward, N, Fs)
    
    # DFT noise
    noise_dft = dft(noise, N, Fs)
    
    #____PLOT____#
    i = np.arange(0,N, dtype=int)

    fig = plt.figure(constrained_layout=True, figsize=(20,8))

    gs = GridSpec(3, 6, figure=fig)
    ax1 = fig.add_subplot(gs[0, :1])
    ax2 = fig.add_subplot(gs[0, 1:2])
    ax3 = fig.add_subplot(gs[1, :1])
    ax4 = fig.add_subplot(gs[1, 1:2])
    ax5 = fig.add_subplot(gs[0, 2:4])
    ax6 = fig.add_subplot(gs[0, 4:])
    ax7 = fig.add_subplot(gs[1, 2:4])
    ax8 = fig.add_subplot(gs[1, 4:])
    ax9 = fig.add_subplot(gs[2, :])
    
    m = np.arange(-M, M+1, 1)
    n = np.arange(-180,181, 1)
    i = np.arange(N)
    j = np.arange(1, int(round(N/2)))
    
    # Impulse Response w(i)
    ax1.stem(m, w[m])
    ax1.set_title('Frequency Impulse w(i)', fontsize='large', color='b')
    ax1.set_xlabel('i samples', fontsize='x-large')
    ax1.grid()
    
    # Frequenxy Response Hw
    ax2.plot((n/180)*np.pi, hw_dft[n],  color='r', linewidth=1.0)
    ax2.set_title('Frequency Response Hw(ohm)', fontsize='large', color='b')
    ax2.set_xlabel('ohm', fontsize='x-large')
    ax2.grid()
    
    # Impulse Response hw(i)
    ax3.stem(m, hw[m])
    ax3.set_title('Impulse Response hw(ohm)', fontsize='large', color='b')
    ax3.set_xlabel('i samples', fontsize='x-large')
    ax3.grid()
    
    # Impulse Response Hw in dB
    ax4.plot((n/180)*np.pi, hw_dft_db[n], color='r', linewidth=1.0)
    ax4.set_title('Impulse Response Hw(ohm) dalam dB', fontsize='large', color='b')
    ax4.set_xlabel('ohm', fontsize='x-large')
    ax4.grid()
    
    # Filtered Signal
    ax5.plot(i,y_backward[i],  color='r', linewidth=1.0)
    ax5.set_title('Sinyal Output', fontsize='xx-large', color='b')
    ax5.set_xlabel('ohm', fontsize='x-large')
    ax5.set_ylabel('Tegangan[V]', fontsize='x-large')
    ax5.grid()
    
    # DFT Filtered Signal
    ax6.plot(j, output_dft[j],  color='r', linewidth=1.0)
    ax6.set_title('Respon Frekuensi Sinyal Output', fontsize='xx-large', color='b')
    ax6.set_xlabel('Frekuensi [Hz]', fontsize='x-large')
    ax6.set_ylabel('Magnitudo', fontsize='x-large')
    ax6.grid()
    
    # Noise
    ax7.plot(i,noise[i],  color='r', linewidth=1.0)
    ax7.set_title('Noise yang dihilangkan', fontsize='xx-large', color='b')
    ax7.set_xlabel('n sample', fontsize='x-large')
    ax7.set_ylabel('Tegangan [V]', fontsize='x-large')
    ax7.grid()
    
    # DFT noise
    ax8.plot(j, noise_dft[j],  color='r', linewidth=1.0)
    ax8.set_title('Respon Frekuensi Sinyal Noise', fontsize='xx-large', color='b')
    ax8.set_xlabel('Frekuensi [Hz]', fontsize='x-large')
    ax8.set_ylabel('Magnitudo', fontsize='x-large')
    ax8.grid()
    
    # Input, output forward, and output backward
    ax9.plot(i,x[i], label='input', color='r')
    ax9.plot(i,y_forward[i], label='forward', color='g')
    ax9.plot(i,y_backward[i], label='backward', color='b')
    ax9.set_title('sinyal input, output forward, output backward', fontsize='xx-large', color='b')
    ax9.set_xlabel('n samples', fontsize='x-large')
    ax9.set_ylabel('Tegangan [V]', fontsize='x-large')
    ax9.legend(fontsize=10, loc=1)
    ax9.grid()

In [63]:
widgets.interact(fir, filter=fil, window=win, M=(1, 200, 1), fc=(1, 500, 1), fl=(1, 500, 1), fh=(1, 500, 1))

interactive(children=(RadioButtons(description='Filter', options=('LPF', 'HPF', 'BPF', 'BSF'), value='LPF'), R…

<function __main__.fir(filter='LPF', window='Rectangular', M=6, fc=30, fl=0, fh=0)>

In [59]:
def zplane(b, a):
    if np.max(b) > 1:
        kn = np.max(b)
        b = b/float(kn)
    else :
        kn = 1

    if np.max(a) > 1:
        kd = np.max(a)
        a = a/float(kd)
    else:
        kd = 1

    # Get the poles and zeros
    p = np.roots(a)
    z = np.roots(b)
    k = kn/float(kd)

    return p, z

In [60]:
orde = widgets.RadioButtons(
    options=['Orde 1', 'Orde 2'],
    value='Orde 1',
    description='Orde'
)

fil = widgets.RadioButtons(
    options=['LPF', 'HPF', 'BPF', 'BSF'],
    value='LPF',
    description='Filter'
)

def iir (ord='Orde 1', filter='LPF', a=0.058, r=0.84, fc=25):
    fs = int(np.round(Fs/2))
    y_forward = np.zeros(2*N+1)
    y_backward = np.zeros(2*N+1)
    noise = np.zeros(2*N+1)
    ohm = np.arange(0, 361)
    H = np.zeros(360)
    phase = np.zeros(360)
    z_real= np.zeros(360)
    z_imag= np.zeros(360)
    p_real = np.zeros(360)
    p_imag = np.zeros(360)
    theta = 2*np.pi*fc/Fs
    
    if ord=='Orde 1' and filter=='LPF':
        K = (1-a)/2 
        
        # Zero-Pole
        top = np.array([1, 1])
        bot = np.array([1, -a])

        p, z = zplane(top, bot)
        
        # Frequency Response
        z_real = 1+np.cos(ohm*np.pi/360)
        z_imag = -np.sin(ohm*np.pi/360)
        p_real = 1-a*np.cos(ohm*np.pi/360)
        p_imag = a*np.sin(ohm*np.pi/360)
        
        H = K*(np.sqrt((z_real**2+z_imag**2)/(p_real**2+p_imag**2)))
        
        # Filtered Signal
        for i in range(N): # forward
            y_forward[i]=a*y_forward[i-1]+K*(x[i]+x[i-1])
            
        for i in range(N): # backward & noise
            y_backward[i]=a*y_backward[i+1]+K*(y_forward[i]+y_forward[i+1])
            noise[i] = x[i]-y_backward[i]

        # Frequency Cut-off
        fc_l = int(np.round(-Fs*(a-1)/(2*np.pi)))
        fc_h = int(np.round(Fs*(np.pi-1-a)/(2*np.pi)))
        print('fc < (fs/4) : {0} Hz \t fc > (fs/4) : {1} Hz'.format(fc_l, fc_h))
    
    elif ord=='Orde 2' and filter=='LPF':
        K = (1-2*r*np.cos(theta)+r**2)/4

        # Zero-Pole
        top = np.array([1, 2, 1])
        bot = np.array([1, -2*r*np.cos(theta), r**2])

        p, z = zplane(top, bot)

        # Frequency Response
        z_real = 1+2*np.cos(ohm*np.pi/360)+np.cos(2*ohm*np.pi/360)
        z_imag = -2*np.sin(ohm*np.pi/360)-np.sin(2*ohm*np.pi/360)
        p_real = 1-2*r*np.cos(theta)*np.cos(ohm*np.pi/360)+(r**2)*np.cos(2*ohm*np.pi/360)
        p_imag = 2*r*np.cos(theta)*np.sin(ohm*np.pi/360)-(r**2)*np.sin(2*ohm*np.pi/360)

        H = K*(np.sqrt((z_real**2+z_imag**2)/(p_real**2+p_imag**2)))

        # Filtered Signal
        for i in range(N): # forward
            y_forward[i]=(2*r*np.cos(theta)*y_forward[i-1]
                          -(r**2)*y_forward[i-2]
                          +K*(x[i]+2*x[i-1]+x[i-2]))
            
        for i in range(N): # backward & noise
            y_backward[i]=(2*r*np.cos(theta)*y_backward[i+1]
                          -(r**2)*y_backward[i+2]
                          +K*(y_forward[i]+2*y_forward[i+1]+y_forward[i+2]))

            noise[i] = x[i]-y_backward[i]
    
        # Frequency Cut-off
        print('theta : {}'.format(theta))
    
    elif ord=='Orde 1' and filter=='HPF':
        K = (1+a)/2

        # Zero-Pole
        top = np.array([1, -1])
        bot = np.array([1, -a])

        p, z = zplane(top, bot)
        
        # Frequency Response
        z_real = 1-np.cos(ohm*np.pi/360)
        z_imag = -np.sin(ohm*np.pi/360)
        p_real = 1-a*np.cos(ohm*np.pi/360)
        p_imag = a*np.sin(ohm*np.pi/360)
        
        H = K*(np.sqrt((z_real**2+z_imag**2)/(p_real**2+p_imag**2)))
        
        # Filtered Signal
        for i in range(N): # forward
            y_forward[i]=a*y_forward[i-1]+K*(x[i]-x[i-1])
            
        for i in range(N): # backward & noise
            y_backward[i]=a*y_backward[i+1]+K*(y_forward[i]-y_forward[i+1])
            noise[i] = x[i]-y_backward[i]

        # Frequency Cut-off
        fc_l = int(np.round(-Fs*(a-1)/(2*np.pi)))
        fc_h = int(np.round(Fs*(np.pi-1-a)/(2*np.pi)))
        print('fc < (fs/4) : {0} Hz \t fc > (fs/4) : {1} Hz'.format(fc_l, fc_h))
        
    elif ord=='Orde 2' and filter=='HPF':
        K = (1+2*r*np.cos(theta)+r**2)/4

        # Zero-Pole
        top = np.array([1, -2, 1])
        bot = np.array([1, -2*r*np.cos(theta), r**2])

        p, z = zplane(top, bot)

        # Frequency Response
        z_real = 1-2*np.cos(ohm*np.pi/360)+np.cos(2*ohm*np.pi/360)
        z_imag = 2*np.sin(ohm*np.pi/360)-np.sin(2*ohm*np.pi/360)
        p_real = 1-2*r*np.cos(theta)*np.cos(ohm*np.pi/360)+(r**2)*np.cos(2*ohm*np.pi/360)
        p_imag = 2*r*np.cos(theta)*np.sin(ohm*np.pi/360)-(r**2)*np.sin(2*ohm*np.pi/360)

        H = K*(np.sqrt((z_real**2+z_imag**2)/(p_real**2+p_imag**2)))

        # Filtered Signal
        for i in range(N): # forward
            y_forward[i]=(2*r*np.cos(theta)*y_forward[i-1]
                          -(r**2)*y_forward[i-2]
                          +K*(x[i]-2*x[i-1]+x[i-2]))
            
        for i in range(N): # backward & noise
            y_backward[i]=(2*r*np.cos(theta)*y_backward[i+1]
                          -(r**2)*y_backward[i+2]
                          +K*(y_forward[i]-2*y_forward[i+1]+y_forward[i+2]))

            noise[i] = x[i]-y_backward[i]
    
        # Frequency Cut-off
        print('theta : {}'.format(theta))


    elif ord=='Orde 2' and filter=='BPF':
        K = (((1-r)*np.sqrt(1-2*r*np.cos(2*theta)+r**2))
            /(2*abs(np.sin(theta))))

        # Zero-Pole
        top = np.array([1, 0, -1])
        bot = np.array([1, -2*r*np.cos(theta), r**2])

        p, z = zplane(top, bot)
        
        # Frequency Response
        z_real = 1-np.cos(2*ohm*np.pi/360)
        z_imag = np.sin(2*ohm*np.pi/360)
        p_real = 1-2*r*np.cos(theta)*np.cos(ohm*np.pi/360)+(r**2)*np.cos(2*ohm*np.pi/360)
        p_imag = 2*r*np.cos(theta)*np.sin(ohm*np.pi/360)-(r**2)*np.sin(2*ohm*np.pi/360)
        
        H = K*(np.sqrt((z_real**2+z_imag**2)/(p_real**2+p_imag**2)))
        
        # Filtered Signal
        for i in range(N): # forward
            y_forward[i]=(2*r*np.cos(theta)*y_forward[i-1]
                          -(r**2)*y_forward[i-2]
                          +K*(x[i]-x[i-2]))
            
        for i in range(N): # backward & noise
            y_backward[i]=(2*r*np.cos(theta)*y_backward[i+1]
                          -(r**2)*y_backward[i+2]
                          +K*(y_forward[i]-y_forward[i+2]))

            noise[i] = x[i]-y_backward[i]

        # Frequency Cut-off
        print('f center : {} Hz'.format(fc))

    elif ord=='Orde 2' and filter=='BSF':
        K = ((1-2*r*np.cos(theta)+r**2)
            /(2-2*np.cos(theta)))

        # Zero-Pole
        top = np.array([1, 2*np.cos(theta), 1])
        bot = np.array([1, -2*r*np.cos(theta), r**2])

        p, z = zplane(top, bot)
        
        # Frequency Response
        z_real = 1-2*np.cos(theta)*np.cos(ohm*np.pi/360)+np.cos(2*ohm*np.pi/360)
        z_imag = 2*np.cos(theta)*np.sin(ohm*np.pi/360)-np.sin(2*ohm*np.pi/360)
        p_real = 1-2*r*np.cos(theta)*np.cos(ohm*np.pi/360)+(r**2)*np.cos(2*ohm*np.pi/360)
        p_imag = 2*r*np.cos(theta)*np.sin(ohm*np.pi/360)-(r**2)*np.sin(2*ohm*np.pi/360)
        
        H = K*(np.sqrt((z_real**2+z_imag**2)/(p_real**2+p_imag**2)))
        
        # Filtered Signal
        for i in range(N): # forward
            y_forward[i]=(2*r*np.cos(theta)*y_forward[i-1]
                          -(r**2)*y_forward[i-2]
                          +K*(x[i]-2*np.cos(theta)*x[i-1]+x[i-2]))
            
        for i in range(N): # backward & noise
            y_backward[i]=(2*r*np.cos(theta)*y_backward[i+1]
                          -(r**2)*y_backward[i+2]
                          +K*(y_forward[i]-2*np.cos(theta)*y_forward[i+1]+y_forward[i+2]))

            noise[i] = x[i]-y_backward[i]

        # Frequency Cut-off
        print('f center : {} Hz'.format(fc))

    elif ord=='Orde 1' and (filter=='BPF' or filter=='BSF') :
        print('Band Pass Filter & Band Stop Filter must in Orde 2!')
    else :
        print('Error!')

    # DFT filtered Signal
    output_dft = dft(y_backward, N, Fs)
    
    # DFT noise
    noise_dft = dft(noise, N, Fs)


    fig = plt.figure(constrained_layout=True, figsize=(20,8))
    
    gs = GridSpec(3, 6, figure=fig)
    ax1 = fig.add_subplot(gs[0, :1])
    ax2 = fig.add_subplot(gs[1, :1])
    ax3 = fig.add_subplot(gs[0, 1:4])
    ax4 = fig.add_subplot(gs[0, 4:])
    ax5 = fig.add_subplot(gs[1, 1:4])
    ax6 = fig.add_subplot(gs[1, 4:])
    ax7 = fig.add_subplot(gs[2, :])
    
    n = np.arange(N)
    i = np.arange(0,361)
    m = np.arange(1, fs)
    
    # Z-Pole
    uc = patches.Circle((0,0), radius=1, fill=False,
                        color = 'black', ls='dashed')
    ax1.add_patch(uc)
    
    zero = ax1.plot(z.real, z.imag, 'go', ms=10)
    plt.setp(zero, markersize=10.0, markeredgewidth=1.0,
             markeredgecolor='k', markerfacecolor='g')
    
    pole = ax1.plot(p.real, p.imag, 'rx', ms=10)
    plt.setp(pole, markersize=12.0, markeredgewidth=3.0,
             markeredgecolor='r', markerfacecolor='r')
                   
    ax1.set_ylim(ymin=-1.5,  ymax=1.5)
    ax1.set_xlim(xmin=-1.5,  xmax=1.5)
    
    ax1.set_title('Z-Pole Plane', fontsize='xx-large', color='b')
    ax1.set_xlabel('Re', fontsize='x-large')
    ax1.set_ylabel('Im', fontsize='x-large')
    ax1.grid()
    
    # Respon Magnitude
    ax2.plot((i/360)*(fs/Fs), H[i], color='r', lw=1.0)
    ax2.set_title('Respon Magnitude |H(f/fs)|', fontsize='x-large', color='b')
    ax2.set_xlabel('Frekuensi normalisasi (f/fs)', fontsize='large')
    ax2.grid()
    
    # Filtered Signal
    ax3.plot(n, y_backward[n], color='r', lw=1.0)
    ax3.set_title('Sinyal Output', fontsize='xx-large', color='b')
    ax3.set_xlabel('n samples', fontsize='x-large')
    ax3.set_ylabel('Tegangan [V]', fontsize='x-large')
    ax3.grid()

    # DFT Filtered Signal
    ax4.plot(m, output_dft[m], color='r', lw=1.0)
    ax4.set_title('Response Frekuensi Sinyal Output', fontsize='xx-large', color='b')
    ax4.set_xlabel('Frekuensi [Hz]', fontsize='x-large')
    ax4.set_ylabel('Magnitudo', fontsize='x-large')
    ax4.grid()
    
    # Noise Signal
    ax5.plot(n, noise[n], color='r', lw=1.0)
    ax5.set_title('Noise yang Dihilangkan', fontsize='xx-large', color='b')
    ax5.set_xlabel('n samples', fontsize='x-large')
    ax5.set_ylabel('Tegangan [V]', fontsize='x-large')
    ax5.grid()

    # DFT Noise Signal
    ax6.plot(m, noise_dft[m], color='r', lw=1.0)
    ax6.set_title('Response Frekuensi Sinyal Noise', fontsize='xx-large', color='b')
    ax6.set_xlabel('Frekuensi [Hz]', fontsize='x-large')
    ax6.set_ylabel('Magnitudo', fontsize='x-large')
    ax6.grid()

    # Input, output forward, and output backward
    ax7.plot(i,x[i], label='input', color='r')
    ax7.plot(i,y_forward[i], label='forward', color='g')
    ax7.plot(i,y_backward[i], label='backward', color='b')
    ax7.set_title('sinyal input, output forward, output backward', fontsize='xx-large', color='b')
    ax7.set_xlabel('n samples', fontsize='x-large')
    ax7.set_ylabel('Tegangan [V]', fontsize='x-large')
    ax7.legend(fontsize=10, loc=1)
    ax7.grid()

  
        

In [61]:
widgets.interact(iir, ord=orde, filter=fil, a=(-0.998, 0.999, 0.001), r=(0, 1, 0.001), fc=(0, 500, 1))

interactive(children=(RadioButtons(description='Orde', options=('Orde 1', 'Orde 2'), value='Orde 1'), RadioBut…

<function __main__.iir(ord='Orde 1', filter='LPF', a=0.058, r=0.84, fc=25)>